## Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [1]:
import pandas as pd

In [2]:
jan_data = pd.read_parquet("./data/yellow_tripdata_2023-01.parquet")

print(f"Jan data has {jan_data.shape[1]} columns")

Jan data has 19 columns


## Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

In [3]:
import datetime as datetime

In [4]:
jan_data["duration"] = jan_data["tpep_dropoff_datetime"] - jan_data["tpep_pickup_datetime"]
duration_std_dev: datetime.timedelta = jan_data["duration"].std()

print(f"Standard deviation of rides is {'{0:.2f}'.format(duration_std_dev.total_seconds() / 60)} minutes")

Standard deviation of rides is 42.59 minutes


## Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [5]:
import matplotlib.pyplot as plt

In [17]:
jan_data["duration_seconds"] = jan_data["duration"].dt.total_seconds()
jan_data["duration_minutes"] = jan_data["duration_seconds"] * (1/60)
row_count = jan_data.shape[0]

# jan_data["duration_minutes"].hist(log=True)
# plt.title("Distribution of durations in mins")
# plt.show()

jan_data_filtered = jan_data[jan_data["duration_minutes"].between(1, 60, inclusive='both')]
filtered_row_count = jan_data_filtered.shape[0]

# plt_2 = jan_data_filtered["duration_minutes"].hist(log=False)
# plt.title("Distribution without outliers")
# plt.show()

print(f"{round(100 * filtered_row_count / row_count)}% of rows remain in the dataset")


98% of rows remain in the dataset


## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [7]:
from sklearn.feature_extraction import DictVectorizer

In [8]:
features = pd.DataFrame()
features["PULocationID"] = jan_data_filtered["PULocationID"].apply(lambda x: str(x))
features["DOLocationID"] = jan_data_filtered["DOLocationID"].apply(lambda x: str(x))
features_dict = features.to_dict(orient="records")

vectorizer = DictVectorizer(sparse=True)
feature_matrix = vectorizer.fit_transform(features_dict)

print(f"The feature matrix has {feature_matrix.shape[1]} columns")

The feature matrix has 515 columns


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [10]:
model = LinearRegression(n_jobs=16)
model = model.fit(feature_matrix, jan_data_filtered["duration_minutes"])

In [11]:
import math

In [12]:
training_predicted = model.predict(feature_matrix)
rmse_training = math.sqrt(mean_squared_error(jan_data_filtered["duration_minutes"], training_predicted))
print(f"rmse of model on training data is {'{0:.2f}'.format(rmse_training)}")

rmse of model on training data is 7.65


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

In [13]:
feb_data = pd.read_parquet("./data/yellow_tripdata_2023-02.parquet")

feb_data["duration"] = feb_data["tpep_dropoff_datetime"] - feb_data["tpep_pickup_datetime"]
feb_data["duration_seconds"] = feb_data["duration"].dt.total_seconds()
feb_data["duration_minutes"] = feb_data["duration_seconds"] * (1/60)
feb_data_filtered = feb_data[feb_data["duration_minutes"].between(1, 60, inclusive='both')]

print(f"{round((feb_data_filtered.shape[0] / feb_data.shape[0]) * 100)}% of rows remain in the dataset")

98% of rows remain in the dataset


In [14]:
feb_features = pd.DataFrame()
feb_features["PULocationID"] = feb_data_filtered["PULocationID"].apply(lambda x: str(x))
feb_features["DOLocationID"] = feb_data_filtered["DOLocationID"].apply(lambda x: str(x))
feb_features_dic = feb_features.to_dict(orient="records")
feb_feature_matrix = vectorizer.transform(feb_features_dic)

In [15]:
feb_predicted = model.predict(feb_feature_matrix)
rmse_test = math.sqrt(mean_squared_error(feb_data_filtered["duration_minutes"], feb_predicted))
print(f"rmse of model for test data is {'{0:.2f}'.format(rmse_test)}")

rmse of model for test data is 7.81
